In [1]:
import numpy as np
import pickle
import gzip

In [2]:
#Write a Data Loader

def load_data(file = "data/mnist.pkl.gz"):
    f = gzip.open(file,"rb")
    u = pickle._Unpickler(f)
    u.encoding = "latin1"
    p = u.load()
    return p

def vectorized_results(j):
    e = np.zeros((10,1))
    e[j] = 1
    return e

def load_data_wrapper():
    tr_d , va_d , te_d = load_data()
    training_inputs = [
        np.reshape(x,(784,1)) for x in tr_d[0]
    ]
    training_results = [
        vectorized_results(y) for y in tr_d[1]
    ]
    training_data = zip(
        training_inputs , training_results
    )
    validation_inputs = [
        np.reshape(x,(784,1)) for x in va_d[0]
    ]
    validation_data = zip(
        validation_inputs , va_d[1]
    )
    test_inputs = [
        np.reshape(x , (784,1)) for x in te_d[0]
    ]
    test_data = zip(test_inputs , te_d[1])
    return list(training_data) , list(validation_data) , list(test_data)

def sigmoid(x):
    return 1/(1 + np.exp(-x))

In [3]:

class BaseNetwork:
    
    def __init__(self , size , cost = None , batch_size = 1):
        self.size = size
        self.num_layers = len(size)
        self.cost = cost
        self.batch_size = batch_size
        self.weights = self.initialize_weights()
        self.biases = self.initialize_biases()
        
    def _get_weight_dimensions(self):
        '''
        Return an iterable with the sizes of weights for each layer
        '''
        return zip(
            self.size[:-1] , self.size[1:]
        )
    def _get_bias_dimensions(self):
        '''
        Return an iterable with the sizes of the biases for each layer
        '''
        return self.size[1:]
    
    def initialize_weights(self):
        '''
        Initialize the weights using Gaussian distribution with mean 0
        and sd 1 over the square root of the number of weights connectin
        to the same neuron
        '''
        sizes = self._get_weight_dimensions()
        
        return [
            np.random.randn(y,x)/np.sqrt(x)
            for x,y in sizes
        ]
    
    def initialize_biases(self):
        size = self._get_bias_dimensions()
        return [
            np.random.randn(self.batch_size , y,1) for y in size
        ]
    
    def activation(self , w , b,a):
        return sigmoid(
            np.matmul(w,a) + b
        )
        
    def inspect(self):
        print("Weights")
        print([np.shape(e) for e in self.weights ])
        print("Biases")
        print([np.shape(e) for e in self.biases ])
        
    def feedforward(self , x):
        a = x
        for w,b in zip(self.weights , self.biases):
            a = self.activation(w,b,a)
        return a
    
    def backprop(self , x , y):
        '''
        Implement the backprop Pipeline
        '''
        return
        

In [4]:
tr_d , va_d , te_d = load_data_wrapper()
net = BaseNetwork([784 , 30 , 10] , batch_size = 10)
net.inspect()

Weights
[(30, 784), (10, 30)]
Biases
[(10, 30, 1), (10, 10, 1)]
